<a href="https://colab.research.google.com/github/lygitdata/aml_project/blob/main/project2/feature_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries

In [8]:
import csv
import os
import neurokit2 as nk
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats
from scipy.signal import find_peaks
from scipy.stats import kurtosis, skew
import multiprocess as mp
from tqdm import tqdm
import hrvanalysis

# Change this path to the folder that has your data
fpath = ""

# Import dataframes

In [2]:
def load_data(train_path, test_path):
    X_train = pd.read_csv(train_path, index_col="id")
    y_train = X_train.iloc[:, 0]
    X_train = X_train.iloc[:, 1:]
    X_test = pd.read_csv(test_path, index_col="id")
    return transform_data(X_train), y_train.values, transform_data(X_test)

def transform_data(df):
    return np.array([row.dropna().to_numpy(dtype='float32') for _, row in df.iterrows()], dtype=object)

In [3]:
X_train_raw, y_train_raw, X_test_raw = load_data(
    train_path = f"{fpath}train.csv",
    test_path = f"{fpath}test.csv"
)
print(
    "X_train_raw shape: ",
    X_train_raw.shape,
    "\ny_train_raw shape",
    y_train_raw.shape,
    "\nX_test_raw shape",
    X_test_raw.shape,
)

X_train_raw shape:  (5117,) 
y_train_raw shape (5117,) 
X_test_raw shape (3411,)


# Feature engineering

In [ ]:
def _feature_engineering(signal, method="neurokit"):
    # Feature vector to store the extracted features
    features = []

    try:
        # Attempt using the first method
        signals, info = nk.ecg_process(
            ecg_signal=signal, sampling_rate=300, method="neurokit"
        )
    except Exception as e:
        print("Method 'neurokit' failed:", e)
        try:
            # Fallback to second method
            signals, info = nk.ecg_process(
                ecg_signal=signal, sampling_rate=300, method="pantompkins1985"
            )
        except Exception as e:
            print("Method 'pantompkins1985' failed:", e)
            try:
                # Fallback to third method
                signals, info = nk.ecg_process(
                    ecg_signal=signal, sampling_rate=300, method="hamilton2002"
                )
            except Exception as e:
                print("Method 'hamilton2002' failed:", e)
                try:
                    # Fallback to fourth method
                    signals, info = nk.ecg_process(
                        ecg_signal=signal, sampling_rate=300, method="elgendi2010"
                    )
                except Exception as e:
                    print("Method 'elgendi2010' failed:", e)
                    try:
                        # Fallback to fifth method
                        signals, info = nk.ecg_process(
                            ecg_signal=signal, sampling_rate=300, method="engzeemod2012"
                        )
                    except Exception as e:
                        print("Method 'engzeemod2012' failed:", e)
                        raise ValueError(
                            "All methods failed. Please check the input signal."
                        )

    # Access relevant outputs from the tuple
    rpeaks = info["ECG_R_Peaks"]
    heart_rate = np.array(signals["ECG_Rate"][info["ECG_R_Peaks"]])

    # Feature summary stats vectors:
    for key in ["ECG_P_Peaks", "ECG_Q_Peaks", "ECG_S_Peaks", "ECG_T_Peaks"]:
        idx = np.array(info[key])[~np.isnan(info[key])]
        features += summary_stats(idx)
        features += summary_stats(np.diff(idx))
        features += summary_stats(np.array(signals["ECG_Clean"][idx.astype(int)]))
        features += summary_stats(np.diff(signals["ECG_Clean"][idx.astype(int)]))

    # PR interval, PR segment, QT interval, ST segment, QRS complex
    p_r_int = np.array(signals["ECG_P_Onsets"] - signals["ECG_R_Onsets"])
    p_r_seg = np.array(signals["ECG_P_Offsets"] - signals["ECG_R_Onsets"])
    q_t_int = np.array(signals["ECG_Q_Peaks"] - signals["ECG_T_Offsets"])
    s_t_seg = np.array(signals["ECG_S_Peaks"] - signals["ECG_T_Onsets"])
    qrs_cpx = np.array(signals["ECG_Q_Peaks"] - signals["ECG_S_Peaks"])
    for item in [p_r_int, p_r_seg, q_t_int, s_t_seg, qrs_cpx]:
        features += summary_stats(item[item != 0])

    # Spectral features using FFT
    clip = signal[rpeaks[0] : rpeaks[-1]]  # Clip signal around R-peaks
    freq = np.fft.rfftfreq(len(clip), 1 / 300)
    spec = np.abs(np.fft.rfft(clip)) / len(clip)
    freq, spec = binned(freq, spec, 50.0, 100, np.max)
    features += list(spec)

    # Autocorrelation of the signal
    autocorr = np.correlate(clip, clip, mode="full") / len(clip)
    autocorr = autocorr[autocorr.size // 2 :]
    time = np.linspace(0, len(clip) / 300, len(clip))
    time, autocorr = binned(time, autocorr, 1.0, 100, np.mean)
    features += list(autocorr)

    # Heart rate features
    features += summary_stats(heart_rate)

    # HRV (Heart Rate Variability)
    rr_intervals = np.diff(rpeaks) / 300
    features += summary_stats(rr_intervals)

    # Signal entropy
    entropy = signal_entropy(clip)
    features.append(entropy)

    # Time-domain features
    features += time_domain_features(clip)

    # New Features
    # 1. Normalized RR intervals
    rr_normalized = (rr_intervals - np.min(rr_intervals)) / (
        np.max(rr_intervals) - np.min(rr_intervals)
    )
    features += summary_stats(rr_normalized)

    # 2. Skewness and Kurtosis of RR intervals
    features.append(skew(rr_intervals))
    features.append(kurtosis(rr_intervals))

    # 3. Peak-to-peak amplitude variability
    r_amplitudes = signal[rpeaks]
    features.append(np.std(r_amplitudes))

    # 4. Median Absolute Deviation (MAD) of Clean ECG
    mad_ecg_clean = np.median(
        np.abs(signals["ECG_Clean"] - np.median(signals["ECG_Clean"]))
    )
    features.append(mad_ecg_clean)

    # 5. Fraction of low-amplitude peaks
    low_amp_fraction = np.sum(r_amplitudes < 0.5 * np.mean(r_amplitudes)) / len(
        r_amplitudes
    )
    features.append(low_amp_fraction)

    # 6. Energy of the signal
    features.append(np.sum(np.square(signal)))

    # 7. Normalized power in frequency bands
    vlf_power = np.sum(spec[(freq >= 0.003) & (freq < 0.04)])
    lf_power = np.sum(spec[(freq >= 0.04) & (freq < 0.15)])
    hf_power = np.sum(spec[(freq >= 0.15) & (freq < 0.4)])
    total_power = vlf_power + lf_power + hf_power
    features += [
        vlf_power / total_power if total_power != 0 else 0,
        lf_power / total_power if total_power != 0 else 0,
        hf_power / total_power if total_power != 0 else 0,
    ]

    # 8. Harmonic mean of RR intervals
    rr_harmonic_mean = len(rr_intervals) / np.sum(1.0 / rr_intervals)
    features.append(rr_harmonic_mean)

    # 9. Baseline wander amplitude
    baseline = nk.signal_filter(signal, sampling_rate=300, lowcut=0.5, method="butter")
    baseline_wander_amplitude = np.max(baseline) - np.min(baseline)
    features.append(baseline_wander_amplitude)

    # 10. Feature extraction using hrvanalysis library
    hrv_features  = extract_hrv_features(rr_intervals*1000)   # Convert RR intervals to milliseconds
    features += list(hrv_features.values())

    # Return the extracted feature vector
    return features


# Function to calculate Shannon entropy of a signal
def signal_entropy(signal):
    prob_density, _ = np.histogram(signal, bins=10, density=True)
    prob_density = prob_density[prob_density > 0]  # Remove zero probabilities
    entropy = -np.sum(prob_density * np.log(prob_density))  # Shannon entropy
    return entropy


# Time-domain statistical features (mean, std, skewness, kurtosis)
def time_domain_features(signal):
    mean = np.mean(signal)
    std = np.std(signal)
    skewness = skew(signal)
    kurt = kurtosis(signal)
    return [mean, std, skewness, kurt]

# Feature extraction using hrvanalysis library

def extract_hrv_features(rr_intervals):
    """
    Extract HRV features using the hrvanalysis package.
    
    Args:
    - rr_intervals: List of RR intervals in milliseconds.

    Returns:
    - features: Dictionary of HRV features.
    """
    try:
        time_features = hrvanalysis.get_time_domain_features(rr_intervals)
        geometric_features = hrvanalysis.get_geometrical_features(rr_intervals)
        frequency_features = hrvanalysis.get_frequency_domain_features(rr_intervals)
        cscv_features = hrvanalysis.get_csi_cvi_features(rr_intervals)
        poincare_features = hrvanalysis.get_poincare_plot_features(rr_intervals)
        
    except:
        return []

    # Combine all features
    features = {**time_features, **frequency_features, **poincare_features, **geometric_features, **cscv_features}
    return features

# Binned function for downsampling and applying a function over binned data
def binned(x, y, xend, nbins, func):
    bx = np.linspace(x[0], xend, nbins + 1)
    idx = np.digitize(x, bx) - 1  # Bin assignments
    bx = (bx[1:] + bx[:-1]) / 2  # Bin centers
    by = np.array(
        [func(y[idx == i]) for i in range(nbins)]
    )  # Apply function to each bin
    return bx, by


# summary_stats function to compute statistical measures
def summary_stats(x):
    x = x[~np.isnan(x)]
    if len(x) == 0:
        return [0, 0, 0, 0]
    else:
        return [np.mean(x), np.std(x), np.median(x), np.var(x)]


# Sequential feature engineering
def feature_engineering(X):
    """
    Sequentially process signals for feature engineering.
    
    Args:
    - X: List or array of ECG signals.

    Returns:
    - X_features: Numpy array of extracted features.
    """
    X_features = []
    for signal in tqdm(X, desc="Feature Engineering"):
        features = _feature_engineering(signal)  # Call your existing feature extraction function
        X_features.append(features)
    return np.array(X_features)

In [20]:
# Do feature engineering
X_train = feature_engineering(X_train_raw)
X_test = feature_engineering(X_test_raw)
print(
    "X_train shape: ",
    X_train.shape,
    "\nX_test shape",
    X_test.shape,
)

Feature Engineering:   1%|          | 58/5117 [00:23<34:38,  2.43it/s]


KeyboardInterrupt: 

In [ ]:
# Save the data as npy to save time for later use
np.save(f"{fpath}feature_extraction/X_train.npy", X_train)
np.save(f"{fpath}feature_extraction/X_test.npy", X_test)
np.save(f"{fpath}feature_extraction/y_train.npy", y_train_raw)